# Import the module

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import analysis as da
import importlib

In [7]:
##################################    
if __name__ == '__main__':
    ################################## 
    importlib.reload(da)
    sdir = "/home/vikasv/Square_Kitaev_vcurrent/files/Jz_0.3/Beta_1/"    
    odir = "/home/vikasv/Square_Kitaev_vcurrent/analysis/"
    mfile = da.run_update(sdir,odir)
    #mfile = da.run_get_file(dir, header)
    ################################## 

Hello, you are extracting *.dat files from  /home/vikasv/Square_Kitaev_vcurrent/files/Jz_0.3/Beta_1/


FileNotFoundError: [Errno 2] No such file or directory: '/home/vikasv/Square_Kitaev_vcurrent/analysis/'

In [29]:
import itertools as it
import numpy as np
from   scipy.sparse import csr_matrix
import scipy as sp
from   scipy.sparse.linalg import eigsh

def Kitaev_Hamiltonian(L, Jxx, Jzz, xbonds, ybonds):
    
    def basis_no(i):
        return sum((2**j) * ((i[L - j - 1] + 1) // 2) for j in range(L))
    
    data = []
    rows = []
    cols = []
    
    for ii in it.product([-1,1], repeat=L):
        # Diagonal elements
        diag_energy = Jzz * 0.5**2 * sum(ii[L - 1 - bond[0]] * ii[L - 1 - bond[1]] for bond in ybonds)
        data.append(diag_energy)
        rows.append(basis_no(ii))
        cols.append(basis_no(ii))
        
        # Off-diagonal elements
        for bond in xbonds:
            flipped_states = [(-ii[L - 1 - bond[0]], -ii[L - 1 - bond[1]])]
            for j in flipped_states:
                jj = list(ii)
                jj[L - 1 - bond[0]] = j[0]
                jj[L - 1 - bond[1]] = j[1]
                jj = tuple(jj)
                data.append(Jxx * 0.5**2)
                rows.append(basis_no(ii))
                cols.append(basis_no(jj))

    Hsp = csr_matrix((data, (rows, cols)), shape=(2**L, 2**L), dtype=np.float64)
    return Hsp



def bond_list(Lx,Ly): # (L = Lx*Ly) valid only for square lattice (Lx==Ly) 
    xbondlist = []
    ybondlist = []
    for i in range(0,Lx*Ly):
        x = int(i/Lx)
        xbondlist.append([i, Lx*x+(i+1)%Ly])
        ybondlist.append([i, (i+Lx)%(Lx*Ly)])           
    return xbondlist, ybondlist 


def Kitaev2D_ED(Lx, Lz, Jxx, Jzz, xbnds, ybnds):
    Hsp = Kitaev_Hamiltonian(Lx*Lz, Jxx, Jzz, xbnds, ybnds)
    if L < 16: Wsp, Vsp = sp.linalg.eigh(Hsp.toarray())
    else: Wsp, Vsp = eigsh(Hsp, k=3, which='SA',return_eigenvectors=True)
    return Wsp, Vsp

# Calculate $<E>$ and $<M^2>$, T = 0.025 

In [31]:
Lx = 4
Lz = 4
L = Lx*Lz
beta = 40

basis_list = list(it.product([-1,1], repeat=L))
Jxx_vals = -1*np.arange(0,1+0.1,0.1)
xbnds, ybnds = bond_list(Lx,Lx)
Energy = []; Magz = [];
for Jxx in Jxx_vals:
    Jzz = -(1-abs(Jxx))
    Wsp, Vsp = Kitaev2D_ED(Lx, Lz, Jxx, Jzz, xbnds, ybnds)    
    e = map(lambda w: w*np.exp(-w*beta), list(Wsp))
    z = map(lambda w: np.exp(-w*beta), list(Wsp))
    print("Gs energy =", Wsp.min()/L - abs(Jxx)/4 - abs(Jzz)/4)
    
    # Energy
    enrg = sum(list(e)/sum(list(z))/L) - abs(Jxx)/4 - abs(Jzz)/4)
    # Magnetization along z
    mzi  = sum([sum(map(lambda sv, ii: 0.5*0.5*abs(sv**2)*abs(np.sum(ii))**2, Vsp[:,i], basis_list)) for i in range(1)])/L
    
    Energy.append(enrg)
    Magz.append(mzi)

Gs energy = -0.5000000000000003
Gs energy = -0.47534818807202994
Gs energy = -0.45158507420998084
Gs energy = -0.4292007826343784
Gs energy = -0.40944772846408684
Gs energy = -0.39853830696646
Gs energy = -0.40944772846408795
Gs energy = -0.42920078263437844
Gs energy = -0.45158507420998
Gs energy = -0.47534818807203083
Gs energy = -0.5


# Import data

In [ ]:
import pandas as pd
df = pd.DataFrame({'Jxx': Jxx_vals, 'Energy': Energy, 'Mag': Magz})
df.to_string("ED_enrg_mag.dat", col_space=[10, 10, 10], sparsify=False, index=None, float_format='%e')     

# Plot of $\langle M^2 \rangle$ Vs $\lambda$

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 24
plt.rcParams['font.serif'] = "computer modern roman"
plt.rcParams['axes.linewidth'] = 1.5

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))


# Assuming it's a CSV file, you can use the read_csv function
df = pd.read_csv("ED_enrg_mag.dat")

ax.plot(Jxx_vals,Magz)
ax.set_xlabel(r'$\lambda$', fontsize = '24', labelpad = 20)
ax.set_ylabel(r'$(M_{z})^2$', fontsize = '26', labelpad = 10)
plt.show()

In [12]:
# function to flip i^th and j^th spins...
def flip(state, i, j):
    state_list=list(state)
    c=state_list[i]
    state_list[i]*= -1
    state_list[j]*= -1
    return (tuple(state_list))



# Function to compute Mz^2 and Mx^2.
def Mag2(basis_list, state_vectors):
    
    Ns = Lx*Lz
    #print(basis_list)
    Mxs = 0   # x2-magnetization
    Mzs = 0   # z2-magnetization
    
    mzi = 0.5*0.5*(state_vector[i]**2)*((np.sum(ii))**2)
    
    for i in range(len(state_vector)):
        ii = basis_list[i]
        Mzs += 0.5*0.5*(state_vector[i]**2)*((np.sum(ii))**2)*(1/(L*L))   # (1/Ns*Ns)*(M_z)^2
        for site1 in range(L):
            for site2 in range(L):
                fstate  = flip(ii,site1, site2)
                j = basis_list.index(fstate)
                Mxs += 0.5*0.5*state_vector[i]*state_vector[j]*(1/(L*L))  # (1/Ns*Ns)*(M_x)^2
            
    return Mxs, Mzs


# Average magnetization at inverse Temp Beta
def avg_mag(Lx, Lz, Jxx, Jzz, s, obc, Beta):
    Ns = Lx*Lz
    H_size = 2**Ns
    
    
    #H = Kitaev2D_ED(Lx, Lz, Jxx, Jzz, s, obc)[0]
    Wsp = Kitaev2D_ED(Lx, Lz, Jxx, Jzz, s, obc)[1]
    Vsp = Kitaev2D_ED(Lx, Lz, Jxx, Jzz, s, obc)[2]
    
    Z = 0
    Mxx = 0
    Mzz = 0
    Eg = 0
    for i in range(H_size):
        state_vector = Vsp[:,i]
        mx,mz =  Mag2(state_vector,Lx,Lz)
        
        Z  += np.exp(-(Wsp[i])*Beta)
        Mxx += np.exp(-(Wsp[i])*Beta) * mx
        Mzz += np.exp(-(Wsp[i])*Beta) * mz
        Eg += np.exp(-(Wsp[i])*Beta) * Wsp[i]
        
    avg_Mx = (Mxx/Z)
    avg_Mz = (Mzz/Z)
    avg_Eg = (Eg/Z)
    
    return avg_Mx, avg_Mz, avg_Eg

-0.41304082349031096

In [47]:

basis_list = list(itertools.product([-1,1], repeat=L))
no_of_states = 2
mzi = sum([sum(map(lambda sv, ii: 0.5*0.5*abs(sv**2)*abs(np.sum(ii))**2, Vsp[:,i], basis_list)) for i in range(no_of_states)])
print(mzi/L)

0.004051749115160791


In [20]:
3 // 2

1